# Control flow

SPDX-License-Identifier: 0BSD

*Note 1:* Some of the questions here use “block” to mean what, in the Python language reference, is called a “suite.” The formal notion of a *block* in Python is related to scoping and differs from the more general usage in this notebook (which matches with how “block” is used in the jargon of most other programming languages). For example, technically there is no such thing as a “`try` block” in Python, but it is common to refer to such, and I do so in this notebook.

*Note 2:* The exercises in this notebook are numbered separately in each section. For example, question 1 in “Decorators review” is separate from and unrelated to question 1 in “General review.”

## Decorators review

### Recursive functions and wrapping decorators

When a wrapping decorator is used on a recursive function definition, the wrapper’s logic accompanies each recursive call to the original function, *not* just the top-level call. For example:

In [1]:
import functools

In [2]:
@functools.cache
def countdown(n):
    if n > 0:
        print(n)
        countdown(n - 1)

In [3]:
countdown(5)

5
4
3
2
1


In [4]:
countdown(3)  # No side effects.

#### Question 1

Explain why the absence of side effects from the above cell demonstrates that, in the cell before that, the `@functools.cache` wrapper’s logic ran for recursive calls, not just the top-level `countdown(5)` call.

#### Answer to question 1

Because if `cache`’s wrapper logic had *not* run recursively, the recursive calls to `countdown` for any `n` less than 5 would not have been cached and subsequent calls to them would have printed.

#### Question 2

Consider the following ***wrong*** argument for the ***false*** claim that decorating a recursive function with a wrapping decorator only affects the top-level call:

> The decorated definition assigns the wrapper to the function name. The top-level call therefore calls the wrapper. The wrapper calls the wrapped function. Since the wrapped function is recursive, it may call itself, and in that call it may call itself, and so forth. Those calls may become quite deep, but crucially, they are all direct recursive calls, from the wrapped function to itself. Therefore there is only one call to the wrapper per top-level call: the top-level call itself.
>
> But the above example shows that the wrapper somehow *does* get called at each level of recursion. I have shown that to be impossible. Yet it still happens. Therefore, nothing is real and we are living in a simulation.

Explain what is wrong with this reasoning.

(If you have trouble, take a bit of time to think about it. If you still have trouble, step through `fibonacci.fibonacci_cached_4` or `fibonacci.fibonacci_cached_5` in the debugger and see what happens.)

#### Answer to question 2

The wrapped function is *not* calling “itself” upon recursive calls, it is calling the *wrapper*, because the wrapped function’s name has been bound to the wrapper.

The top-level call is to the wrapper (which has been assigned to the function name due to decoration), which then calls the wrapped function object (usually passed to the decorator as an argument such as `func` and captured by the wrapper). The call to `func` (`f`) then has a call to the original function’s name (which was bound to wrapper), and thus calls wrapper.

Therefore, adding the decorator converts direct recursion (where a function (`f`) calls itself  `f` → `f` → `f` → `f` ...) to mutual recursion, where (in this case) the wrapper function (`w`) calls the wrapped function (`f`), which calls `w`, and so on (creating a call stack like this `w` → `f` → `w` → `f` ...). 

#### “Question” 3

If you didn’t need to step through `fibonacci.fibonacci_cached_4` or `fibonacci.fibonacci_cached_5` in the debugger, please do so now, since it’s valuable in its own right even if you’ve already correctly answered question 2.

#### Question 4

`fibonacci.fibonacci_cached_4` has a skipped doctest commented, “Mutual-recursion RecursionError.” Explain. 

(If your answer to question 2 did not talk about mutual recursion and use the phrase “mutual recursion” to describe it, revise it so it does. If that is more than a small revision, the answer may not have been correct.)

#### Answer to question 4

`fibonacci.fibonacci_cached_4` is decorated by `memoize`, which wraps. Thus we have mutual recursion and a call to `fibonacci.fibonacci_cached_4(1200)`. The first 500 are stored (see question 5 below), and thus we reach a call depth of around 2 (1200 - 500) = 2 (700) = 1400, which is more than the usual limit of 1000, thus giving us `RecursionError`.

#### Question 5

Before that skipped doctest, there is a non-skipped doctest commented, “No RecursionError: we have up to n=100.” Explain.

#### Answer to question 5

When we define `fibonacci.fibonacci_cached_4` with the decorator `memoize`, we are creating a globally shared cache. Therefore, the running of the first doctests stores the first 100 results in the cache, limiting the recursion depth do it doesn't go over the usual 1000 limit when calling with 500, despite the mutual recursion. Without having the first 100 stored, we would reach 1000 (since we have a call to 500). However, since we already have the first 100 stored, it is as if 100 is a base case. We are calling it with 500, and since 500 - 100 = 400, we reach a depth of about 400 * 2 = 800.

#### Question 6

`fibonacci.fibonacci_cached_5` has a non-skipped doctest commented, “No RecursionError, we split the paths.” Explain.

#### Answer to question 6

The key issue is the longest path in the call tree. 

The helper function `fibonacci.fibonacci_cached_5` contains the recursive statement `return helper(n - 2) + helper(n - 1)`. Thus, upon a call with `n` as 500, the function calls helper with `n` of 498, then 496, and so on. *All* of this must complete *before* it gets to the `+`. Once we get past the plus, and reach `helper(499)` at the top level, *most* results are *already* stored in the cache (everything from 498 down to 0). 

The actual returns are going to happen “bottom-up” and thus the first call which returns, the call to `helper(0)`, called from the first call to `helper(2)`, returns 0. It took about 500/2 (250) calls to the helper function to get here, then it calls `helper(1)` (also a base case) and returns 1, allowing `helper(2)` to be computed. 

However, if we instead had the recursive statement `return helper(n - 1) + helper(n - 2)`, a call with n = 500 would call helper with n = 499, which would call helper with n = 498, and so on. Thus, the first computation that could theoretically return, the call to `helper(1)` from `helper(2)`, would be after about 500 calls to the helper function. That would be a call depth of 1000, due to mutual recursion.   

#### Question 7

There are two major differences between `fibonacci.fibonacci_cached_4` and `fibonacci.fibonacci_cached_5`. One was central to your answer to question 6. The other was irrelevant to it, but very important in general. Explain that difference and its ramifications. Refer to a previous question in this notebook that relates to it.

#### Answer to question 7

`fibonacci.fibonacci_cached_4` has a globally shared cache whereas `fibonacci.fibonacci_cached_5` does not. The reason for this is that the `@memoize` decorator is applied to the top-level function itself in `fibonacci.fibonacci_cached_4`, but only to a local helper function in `fibonacci.fibonacci_cached_5`. Each time there is a call to `fibonacci.fibonacci_cached_5`, the helper function (and its cache) is newly created, whereas in `fibonacci.fibonacci_cached_4`, the cache is created when the function is defined, and shared across calls.

`fibonacci.fibonacci_cached_4`'s behavior is also explained in the answer to question 5.      

#### Question 8

Write a wrapping decorator whose wrapper, on each call, reports that it was called before doing anything else. Use it to decorate the definition of a recursive function that, on each call, reports that it was called before doing anything else. Make sure the original recursive function, and the wrapper, report calls in ways that cannot be confused with each other. Make a top-level call and observe the mutually recursive relationship.

#### Answer to question 8

In [5]:
def reports(func): 
    @functools.wraps(func)
    def wrapper(arg):
        print(f"Wrapper called with {arg}")
        func(arg)
    return wrapper

In [6]:
@reports
def final_countdown(start): 
    print(f"THE FINAL {start}.")
    if start: 
        final_countdown(start - 1)

In [7]:
final_countdown(3)

Wrapper called with 3
THE FINAL 3.
Wrapper called with 2
THE FINAL 2.
Wrapper called with 1
THE FINAL 1.
Wrapper called with 0
THE FINAL 0.


#### Question 9

Create a demonstration like in question 8, but neither function should print or have any other side effect. Instead, define the recursive function so it raises an exception when it reaches some depth. One way to do this is to write a function like `countdown` that doesn’t print and that raises an exception such as `ValueError` in its base case. Another is to write a recursive function with no base case; then `RecursionError` will be raised when excessive depth is reached. (There are other ways, too.) Make a top-level call. This will show how decorating recursive functions with wrapping decorators creates mutual recursion, because you’ll see the alternating calls in the stack trace shown in the error message.

#### Answer to question 9

In [8]:
def no_reports(func): 
    @functools.wraps(func)
    def wrapper(arg):
        func(arg)
    return wrapper

In [9]:
@no_reports
def bomb_timer(seconds_left): 
    if seconds_left == 1: 
        raise ValueError("Don't Brian D it.")
    bomb_timer(seconds_left - 1)

In [10]:
bomb_timer(10)

ValueError: Don't Brian D it.

#### Question 10

Suppose you have a wrapping decorator you could apply to a recursive function definition. You’re writing a recursive function definition, and you *would* decorate it with that decorator... but you only *want* top-level calls to call the wrapper. That is, on this function, for recursive calls, you *want* the wrong claim presented in question 2 to be true! Show how this can be done, by doing it. Use a wrapping decorator that already exists (you will probably want to use the one you wrote for question 8). Write a new `def` statement to create your recursive function. Achieve the goal in a way that does not involve writing any other `def` statements, does not involve `lambda` or `class` or library facilities, and is very simple and clear. Make sure to write code showing that only top-level calls invoke the wrapper.

#### Answer to question 10

In [11]:
def _do_weird_countdown(start): 
    print(f"weird {start}.")
    if start: 
        _do_weird_countdown(start - 1)

weird_countdown = reports(_do_weird_countdown)

In [12]:
weird_countdown(3)

Wrapper called with 3
weird 3.
weird 2.
weird 1.
weird 0.


## General review

**Q1**: Does a `for` loop introduce a scope? Write code that demonstrates that it does or does not.

No:

In [13]:
element = 3
for element in range(10): 
    pass
print(element) 

9


In [14]:
for thing in range(10): 
    pass
print(thing)

9


**Q2**: Does a comprehension introduce a scope? Write code that demonstrates, using a list comprehension, that it does or does not.

Yes: 

In [15]:
element = 3
[element for element in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
print(element)

3


**Q3**: Python has no `do`...`while` loop. Show how that logic can be expressed in Python.

In [17]:
x = 0
print(x)
x += 1
while x < 4: 
    print(x)
    x += 1

0
1
2
3


**Q4**: One way to express the logic of a `do`...`while` loop, which is *occasionally* reasonable, involves repeated code. If that's what you showed above, show the more generally suitable way that does not repeat code. Otherwise, show the way that repeats code.

In [18]:
x = 0 
while True: 
    print(x)
    x += 1
    if x >= 4: 
        break

0
1
2
3


**Q5**: If Python had a `do`...`while` construct, would that help to make the implementation of `functions.as_iterator_limited_alt` simpler than with `while`? Why or why not?

No. If Python had `do`...`while`, someone might write this: 

```python
do: 
    result = func()
    yield result
while result != end_sentinel
```

The code above like it would simplify `functions.as_iterator_limited_alt`, but it does not, because this code doesn’t behave the same way. 

This yields the result *regardless* of whether or not it is the `end_sentinel`, and the correct code does not yield any result if it is the `end_sentinel` because of where the check happens. 

**Q6**: There is a technique we’ll cover later that facilitates simplifying `functions.as_iterator_limited_alt` by using a form of assignment other than an assignment statement (that is, `result = func()` would not appear at all). However, it is reasonable to use an assignment statement, and there are two reasonable ways to do it. Show both of them here, describe how they relate to your answers to previous questions here, and pick whichever of them you like better to use there.

In [19]:
def as_iterator_limited_alt_1(func, end_sentinel):
    result = func()
    while result != end_sentinel:
        yield result
        result = func()

In [20]:
def as_iterator_limited_alt_2(func, end_sentinel): 
    while True: 
        result = func()
        if result == end_sentinel: 
            break
        yield result

`as_iterator_limited_alt_1` repeats code, similar to the implementation of `do`...`while` loop which repeats code in my answer to question 3. `as_iterator_limited_alt_2` does not repeat code, similar to my answer to question 4.  

**Q7**: `pass` is an unusual control flow keyword that most languages don’t have. What does `pass` do? Why does Python benefit from having it, while most programming languages would not benefit (or not as much)?

`pass` does nothing. Python benefits from having it because whitespace is syntactically significant. Sometimes loops, functions, and/or classes require your block to have something in it, even if it does nothing. Other languages (like C and C++) usually surround blocks braces `{}` or require statements to have an ending character (like the dreaded `;`) and therefore a pass statement is unnecessary. Or they may require something like `end` to be written to end a block, which likewise facilitates writing empty blocks.

**Q8**: `pass` doesn’t generally do the same thing as `return`, but in one of its most common uses, replacing it with `return` does not affect the behavior of the code. Show that use. Then show a use where replacing it with `return`, or with any other control flow keyword, behaves differently.

In [21]:
def do_nothing_one(): 
    pass

In [22]:
def do_nothing_two(): 
    return 

In [23]:
do_nothing_one()

In [24]:
do_nothing_two()

In [25]:
def stop_at_4_1():
    for x in range(1, 5): 
        print(x)
        pass      # Does nothing

In [26]:
def stop_at_4_2():
    for x in range(1, 5): 
        print(x)
        return    # !!!!! OH NOOOZ

In [27]:
stop_at_4_1()

1
2
3
4


In [28]:
stop_at_4_2()

1


**Q9**: Does `return` in a `try` block immediately transfer control to an associated `finally` block? Show code that is either an example of this or a demonstration of why it does not occur. (Make sure it behaves differently than if `return` were absent.)

Follow-up question: If control is not transferred to the `finally` block, does that mean a `finally` block—and thus the cleanup it seeks to ensure—can be suppressed by returning directly from a `try` block? (If control *is* transferred to the `finally` block, then answer hypothetically.)

Yes: 

In [29]:
def returns_3(): 
    try: 
        return 3
    finally: 
        print("Got Here")

In [30]:
returns_3()

Got Here


3

If `finally` didn’t transfer control then returning directly *would* suppress the cleanup. However, as stated above, this is *not* how it works, and you can’t suppress the cleanup in this way since the return directly transfers control to the `finally` block. 

**Q10**: Does `raise` in a `try` block immediately transfer control to an associated `finally` block? Show code that is either an example of this or a demonstration of why it does not occur.

Sometimes: if an un-caught exception is raised, the `finally` block runs before the exception propagates out. However, the whole point of `finally` in the `try`-`except`-`else`-`finally` framework is that `finally` is what runs *after* everything, and thus if an exception is caught, or if there is an `else` block, those blocks run first.  

In [31]:
def raise_verror(): 
    try: 
        raise ValueError
    finally: 
        print("Got Here")

In [32]:
raise_verror()

Got Here


ValueError: 

In [33]:
def raise_w_except(): 
    try: 
        raise ValueError
    except ValueError: 
        print("Caught ValueError")
    finally: 
        print("Finally block")

In [34]:
raise_w_except()

Caught ValueError
Finally block


In [35]:
def noexcept_else_finally(): 
    try: 
        print("Try block")
    except ValueError: 
        print("Caught ValueError")
    else: 
        print("No ValueError: Else block")
    finally: 
        print("Finally block")

In [36]:
noexcept_else_finally()

Try block
No ValueError: Else block
Finally block


**Q11**: Can `break` in a `try` block immediately transfer control to an associated `finally` block? Show code that is either an example of this or a demonstration of why it cannot occur. (Make sure it behaves differently than if `break` were absent. Also make sure it behaves differently than if the code in the `try` block after `break` were absent. This might involve guarding the `break` with `if`.)

Yes: 

In [37]:
def break_if_arg_1(arg): 
    for x in range(3):
        try: 
            print(f"{x=} {arg=}")
            if arg == 1: 
                break
            print("Not yet at finally")
        finally: 
            print("Finally block")

In [38]:
break_if_arg_1(0)

x=0 arg=0
Not yet at finally
Finally block
x=1 arg=0
Not yet at finally
Finally block
x=2 arg=0
Not yet at finally
Finally block


In [39]:
break_if_arg_1(1)

x=0 arg=1
Finally block


**Q12:** Can `continue` in a `try` block immediately transfer control to an associated `finally` block? Show code that is either an example of this or a demonstration of why it cannot occur. (Make sure it behaves differently than if `continue` were absent. Also make sure it behaves differently than if the code in the `try` block after `continue` were absent. This might involve guarding the `continue` with `if`.)

Yes: 

In [40]:
def continue_if_arg_1(arg): 
    for x in range(3):
        try: 
            print(f"{x=} {arg=}")
            if arg == 1: 
                continue
            print("Not yet at finally")
        finally: 
            print("Finally block")

In [41]:
continue_if_arg_1(0)

x=0 arg=0
Not yet at finally
Finally block
x=1 arg=0
Not yet at finally
Finally block
x=2 arg=0
Not yet at finally
Finally block


In [42]:
continue_if_arg_1(1)

x=0 arg=1
Finally block
x=1 arg=1
Finally block
x=2 arg=1
Finally block


**Q13:** The most common ways to attempt to quit a Python program are calling `exit` or `quit`, which is best only done in a REPL (because in rare cases they are unavailable), or calling `sys.exit`, which is the recommended way for general use. Does attempting to quit a program by calling one of those functions in a `try` block immediately transfer control to an associated `finally` block? The answer is the same whichever of those three functions you use. Show code that is either an example of this or a demonstration of why it does not occur. Use `sys.exit` in this code.

Yes: 

In [43]:
import sys

In [44]:
def bail():
    try: 
        sys.exit()
    finally: 
        print("Finally block.")

In [45]:
bail()

Finally block.


SystemExit: 

C:\Users\ek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\Local\pypoetry\Cache\virtualenvs\palgoviz-lwOIxvvJ-py3.11\Lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Q14**: To better demonstrate the above, write code with a very similar or identical effect (in particular, that likewise tries to quit), but that does not call `exit`, `quit`, or `sys.exit`.

In [46]:
def raise_systemexit():
    try: 
        raise SystemExit
    finally: 
        print("Finally block.")

In [47]:
raise_systemexit()

Finally block.


SystemExit: 

**Q15**: Can `assert` in `try` block immediately transfer control to an associated `finally` block? Show code that is either an example of this or a demonstration of why it cannot occur.

Yes: 

In [48]:
def assert_falsy(): 
    try: 
        assert 0
        print("Not yet at finally block.")
    finally: 
        print("Finally block.")

In [49]:
assert_falsy()

Finally block.


AssertionError: 

**Q16**: To better demonstrate the above, write code with a similar effect, checking a condition and responding the same way `assert` does if the condition is false, without using `assert`. (But it is okay if your message lacks the details `assert` provides.)

In [50]:
def my_assert(condition, message=None):
    if condition: 
        return 
    if message is None: 
        raise AssertionError()
    raise AssertionError(message)

In [51]:
try:
    assert False
except AssertionError as error:
    print(error.args)

()


In [52]:
try:
    my_assert(False)
except AssertionError as error:
    print(error.args)

()


In [53]:
try:
    my_assert(False, "My message")
except AssertionError as error:
    print(error.args)

('My message',)


In [54]:
def manual_assert_falsy(): 
    try:  
        my_assert(0)
        print("Not yet at finally block.")
    finally: 
        print("Finally block")

In [55]:
manual_assert_falsy()

Finally block


AssertionError: 

**Q17**: The answer to *at least* two of Q9-Q16 was yes. Thus, there are at least two examples above of a control flow statement in a `try` block transferring control directly to a `finally` block. In those examples (however many there were), was the usual effect of that control flow statement suppressed, or did it occur? If it did occur, when did it occur?

No, in all of the examples, the usual effect of the control flow statement was not suppressed, but simply occurred after the code in the finally block ran. This can be seen by looking at my answers and noting that after “finally block” the code ran as if the finally block wasn’t there. 

**Q18**: *This relates closely to Q17 above.* When control is transferred to a `finally` block on its way somewhere else, it is possible for the `finally` block to suppress the transfer to the “intended” destination. It is also possible, and more common, for the transfer not to be suppressed. Most likely, only one of those two claims has been demonstrated in this notebook so far. Write code to demonstrate the other.

In [56]:
def attempt_to_return_3(): 
    try: 
        return 3
    finally: 
        raise ValueError()

In [57]:
attempt_to_return_3()

ValueError: 

In [58]:
def attempt_to_return_3_noexcept(): 
    try: 
        return 3
    finally: 
        return 5

In [59]:
attempt_to_return_3_noexcept()

5

In [60]:
def attempt_to_raise_ValueError():
    try:
        raise ValueError
    finally:
        raise TypeError  # Uh oh.

In [61]:
attempt_to_raise_ValueError()

TypeError: 

**Q19**: Usually a programmer does not wish a `finally` block to suppress transfer of control from a `try` block to somewhere other than the `finally` block. That is, when control goes from a `try` block to a `finally` block, the programmer usually wants it to *subsequently* transfer from the `finally` block to wherever it would’ve gone if there had been no `finally` block. Explain why this is, and give a general guideline that should usually be followed when writing `finally` blocks.

The reason is `finally` is primarily for cleanup, so not subsequently transferring from the `finally` block to the intended destination is itself usually a bug, or can lead bugs that are hard to spot.

General guideline: Don’t raise an exception or return in a `finally` block. This kind of thing can also happen with `break` and `continue`. 

**Q20**: Usually you should not write finalizers (`__del__` methods), but when you do, there are some things you should almost always avoid doing in them. One of those things relates to the insight you just expressed about `finally` blocks. What is that?

You should not raise exceptions in a finalizer. [Exceptions are ignored, and a warning is printed instead.](https://docs.python.org/3/reference/datamodel.html#object.__del__)

If exceptions are raised from either a `finally` block or were raised from a `__del__` method, control would leaving from something that itself is supposed to be cleanup. As shown above, this can lead to some nasty bugs with `finally`, since you may be have unintended exceptions raised instead of intended ones. If `__del__` allowed exceptions to propagate out, the situation would be much worse. This is because `__del__` can run at any time and it would be difficult to debug code if any exception could be raised at any time. 

**Q21**: Some of the insights above are (or were) relevant to simplifying `functions.count_tree_nodes_instrumented`. Explain.

One key insight was understanding how decorators behave with respect to the name of a decorated function. When decorating a function, the name is bound to the function returned by the decorator. Thus, when called recursively, the recursive calls will act with the decorated behavior. This is similar to calling the decorator manually and re-assigning the result to the original name, as we do in the in statement `count_tree_nodes = peek_return(count_tree_nodes)`. This produces an effect similar to decorating the original definition. In question 10 under “Decorators review,” we notably *don't* do this for the `weird_countdown` function, and thus only the first call is wrapped by the wrapping decorator `reports`.

Another key insight was that control is immediately transferred to the `finally` block before going to the intended destination from a `try` block, including upon an exception. This made it safe to write `return count_tree_nodes(root)` in the `try` block rather than storing the return value.

**Q22**: Write a class whose instances are context manager objects that, when exited, print a message. (This will soon be useful for showing when a `with` statement performs cleanup, i.e., for showing when it exits the context manager.)

In [62]:
class PrintsWhenExit: 
    
    def __enter__(self):
        pass
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting")

In [63]:
with PrintsWhenExit() as pwe:
    print("doing stuff.")

doing stuff.
Exiting


**Q23**: Does `return` in a `with` statement cause the context manager to be exited? Show code that is either an example of this or a demonstration of why it does not occur. (Make sure it behaves differently than if `return` were absent.) Also, this behavior is the same as, or different from, the behavior you showed in Q9 above. From the perspective of language design, explain that similarity/difference.

Yes: 

In [64]:
def f(): 
    print("Beginning of f")
    with PrintsWhenExit(): 
        return
    print("End of f")

In [65]:
f()

Beginning of f
Exiting


This behavior is the same as the behavior shown in Q9 above. This similarity makes sense because a `finally` block is conceptually similar to `__exit__`, in that they are both performing needed cleanup.  

**Q24**: Does `raise` in a `with` statement cause the context manager to be exited? Show code that is either an example of this or a demonstration of why it does not occur.

Yes: 

In [66]:
def f(): 
    print("Beginning of f")
    with PrintsWhenExit(): 
        raise ValueError
    print("End of f")

In [67]:
f()

Beginning of f
Exiting


ValueError: 

**Q25**: Can `break` in a `with` statement cause the context manager to be exited? Show code that is either an example of this or a demonstration of why it cannot occur. (Make sure it behaves differently than if `break` were absent. Also make sure it behaves differently than if the code in the `with` statement after `break` were absent. This might involve guarding the `break` with `if`.)

Yes: 

In [68]:
def break_if_arg_1(arg): 
    for x in range(3):
        with PrintsWhenExit(): 
            print(f"{x=} {arg=}")
            if arg == 1: 
                break
            print("End of loop iteration.")

In [69]:
break_if_arg_1(0)

x=0 arg=0
End of loop iteration.
Exiting
x=1 arg=0
End of loop iteration.
Exiting
x=2 arg=0
End of loop iteration.
Exiting


In [70]:
break_if_arg_1(1)

x=0 arg=1
Exiting


**Q26**: Can `continue` in a `with` statement cause the context manager to be exited? Show code that is either an example of this or a demonstration of why it cannot occur. (Make sure it behaves differently than if `continue` were absent. Also make sure it behaves differently than if the code in the `with` statement after `continue` were absent. This might involve guarding the `continue` with `if`.)

Yes:

In [71]:
def cont_if_arg_1(arg): 
    for x in range(3):
        with PrintsWhenExit(): 
            print(f"{x=} {arg=}")
            if arg == 1: 
                continue
            print("End of loop iteration.")

In [72]:
cont_if_arg_1(0)

x=0 arg=0
End of loop iteration.
Exiting
x=1 arg=0
End of loop iteration.
Exiting
x=2 arg=0
End of loop iteration.
Exiting


In [73]:
cont_if_arg_1(1)

x=0 arg=1
Exiting
x=1 arg=1
Exiting
x=2 arg=1
Exiting


**Q27**: Does attempting to quit a program in a `with` statement cause the context manager to be exited? (That is, does it cause the context manager’s cleanup logic to run?) Show code that is either an example of this or a demonstration of why it does not occur.

Yes:

In [74]:
import sys

In [75]:
def bail():
    with PrintsWhenExit():
        sys.exit()

In [76]:
bail()

Exiting


SystemExit: 

**Q28**: Can `assert` in a `with` statement cause the context manager to be exited? Show code that is either an example of this or a demonstration of why it cannot occur.

Yes: 

In [77]:
def assert_falsy(): 
    with PrintsWhenExit(): 
        assert 0

In [78]:
assert_falsy()

Exiting


AssertionError: 

**Q29**: Make a generator function that has a `yield` statement followed by some other statement, together in a `try` block that has an associated `finally` block. Show how the `yield` statement can yield, yet when control reenters the generator, it is transferred immediately to the `finally` block (skipping the statement that followed the `yield` in the `try` block).

In [79]:
def gen(): 
    try: 
        yield 1
        print("Some other statement.")
    finally: 
        print("Finally block.")

In [80]:
import contextlib

In [81]:
with contextlib.closing(gen()) as g: 
    print(next(g))

1
Finally block.


**Q30**: Make a generator function that has a `yield` statement followed by some other statement, together in a `with` statement. Show how the `yield` statement can yield, yet when control reenters the generator, it is transferred immediately to the cleanup logic supplied by the context manager (skipping the statement that followed the `yield` in the `with` block).

In [82]:
def gen_2(): 
    with PrintsWhenExit(): 
        yield 1
        print("Some other statement.")

In [83]:
import contextlib

In [84]:
with contextlib.closing(gen_2()) as g: 
    print(next(g))

1
Exiting


**Q31**: Take your code from Q26 and write it here, but refactor it by extracting the `with` statement to a helper function. The helper shouldn’t just have the suite (“body”) of the `with` statement, but the *entire* `with` statement. But also make sure you don’t extract *more* than the `with` statement. When this is done, `with` will only appear in the helper function, and *the helper function will contain no loops*. Make sure the refactored code runs and behaves the same.

In [85]:
def cont_if_arg_1(arg): 
    def helper(x): 
        with PrintsWhenExit(): 
            print(f"{x=} {arg=}")
            if arg == 1: 
                return
            print("End of loop iteration.")

    for x in range(3):
        helper(x)

In [86]:
cont_if_arg_1(0)

x=0 arg=0
End of loop iteration.
Exiting
x=1 arg=0
End of loop iteration.
Exiting
x=2 arg=0
End of loop iteration.
Exiting


In [87]:
cont_if_arg_1(1)

x=0 arg=1
Exiting
x=1 arg=1
Exiting
x=2 arg=1
Exiting


**Q32**: Q31 illustrates a connection between two control flow keywords, and a related connection between Q26 and another exercise. Explain.

Q31 shows that `continue` becomes `return` when extracting the body of a loop to a helper function. Additionally, this shows how Q23 connects to Q26. Both `return` and `continue` in a `with` statement immediately invoke the `__exit__` method of the context manager. If this were not the case, this transformation would produce different behavior.